# Setup Work Environment

Run the following cell to setup all the dependencies and the code. There should be no changes required from this cell.

Don't forget to ensure the GPU has already attached to your working environment. You can check it in `Runtime -> Manage sessions -> Check if word GPU is available next to the notebook's name`, you can also double check in `Runtime -> Change runtime type -> Check if GPU has already selected from the dropdown menu`

In [ ]:
!rm -rf xib
!pip install pytrie enlighten colorlog inflection ipapy
!git clone https://github.com/akurniawan/xib.git
!cd xib && git clone https://github.com/j-luo93/dev_misc.git && cd dev_misc && git checkout b44fde842a6311e03f731cd4e110dcd9fc394db7 && pip install -e .
!cd xib && pip install -e .

     |████████████████████████████████| 102kB 5.8MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 81kB 6.1MB/s 
  Created wheel for pytrie: filename=PyTrie-0.4.0-cp37-none-any.whl size=6089 sha256=4903b4ffa3a3ec9e56732883a5385d3d9b3f8a90bde2e290435ea15439824553
  Stored in directory: /root/.cache/pip/wheels/ef/2a/41/5870cad27097f3b3d7b3d96aa5897d502db08cafba9051bd62
  Created wheel for ipapy: filename=ipapy-0.0.9.0-cp37-none-any.whl size=38724 sha256=55cbe8d6f98421ec5ac10f9d7b4275416bbf8a775f882be9a12c2ed1b999ccc5
  Stored in directory: /root/.cache/pip/wheels/d1/f3/00/9f83f4dada00246acda572d4248dc8215c5a8ca37bb4f173a8
Successfully built pytrie ipapy
Cloning into 'xib'...
remote: Enumerating objects: 3093, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 3093 (delta 32), reused 47 (delta 24), pack-reused 3032
Receiving objects: 100% (3093/3093), 636.08 KiB | 5.83 MiB/s, 

# Setup Dataset

There are 2 ways to setup your dataset:
1. Mount Gdrive to Colab environment. If you decided to go with this and would like to access the data directly in our `LCT Project` shared folder, you can follow instruction in https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab to load `Preprocessed file` folder in Shared google drive. Essentially you just have to go to the location of the folder, right click and choose `Add a shortcut to Drive`. After that you just have to run the cell below
2. Upload your dataset to `sample_data` folder in google colab environment. Be aware that you **will** lose your data in this folder when you restart colab's environment



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set your dataset path here
KNOWN_LANG_PATH = "/content/sample_data/it_testing.txt"
UNKNOWN_LANG_PATH = "/content/sample_data/it_testing.txt"

# Run Training

As of now, the training will run indefinitely and not sure if changing this will affect the rest of the code. For that reason, we need to stop the training manually once we feel that the losses are no longer improving. We can monitor the `ll` variable from script output inside a table with the following format

```
+----------------------------------------+  
|                  3_8                   |  
+-----------+----------+--------+--------+  
| name      | value    | weight | mean   |  
+-----------+----------+--------+--------+  
| grad_norm | 58.492   | 60     | 0.975  |  
| ll        | -336.246 | 60     | -5.604 |  
| reg       | 0.555    | 60     | 0.009  |  
+-----------+----------+--------+--------+
```

In [ ]:
# Total number of phonetic feature groups
NUM_FEATURE_GROUPS = 10

# Total number of phonetic features
NUM_FEATURES = 10

# Initial value of threshold to determine whether two words are matched. This will determine
# whether two words are in match. The bigger the value, the more false positive we will have.
# However, if the value is too low, the model will not output anything
THRESHOLD = 1.5

# Cost in doing insertion and deletion operation in edit distance algorithm, refer to the paper for more details
INS_DEL_COST = 100.0

# Learning rate for adam optimizer
LR = 0.002

# How many training steps to do before running the evaluation steps
EVAL_INTERVAL = 500

In [ ]:
!PYTHONPATH=/content/xib && /usr/local/bin/python -m xib.main --task extract \
  --vocab_path {KNOWN_LANG_PATH} --data_path {UNKNOWN_LANG_PATH} \
  --dim 112 --min_word_length 1 --max_word_length 10 --input_format text \
  --dense_input --eval_interval 20 \
  --char_per_batch 128 --gpus 0 \
  --num_feature_groups {NUM_FEATURE_GROUPS} --num_features {NUM_FEATURES} \
  --init_threshold {THRESHOLD} --init_ins_del_cost {INS_DEL_COST} --learning_rate {LR}

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/content/xib/dev_misc/dev_misc/utils.py:36: DeprecationWarning: Class/function Trainer deprecated.
  warnings.warn(message, warning_cls)
/content/xib/dev_misc/dev_misc/utils.py:36: RuntimeWarning: Class/function DecipherEsNoisyItalianP5Test is buggy.
  warnings.warn(message, warning_cls)
/usr/local/lib/python3.7/dist-packages/ipapy/ipastring.py:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableSequence
/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
INFO - 05/29/21 09:

# Analyzing The Result

The result will be in the following file `log/<DATE>/default/<TIME>/predictions/extract.<EPOCH>_<STEPS>.tsv`

For some reason, google colab won't show anything under `log` folder, so I would suggest to analyze it via `cat` command or download the result to your local computer and analyze it from there.

The content inside of the `tsv` file will consist of 4 different columns: `segment`, `ground_truth`, `prediction`, `matched_segment`. From my understanding, `segment` is the original segment of the unknown language; `ground_truth` similar to `segment` but with their exact index locations; `prediction` is the vocabulary prediction in the known language; and `matched_segment` is the information on which segment the unknown language match the vocabulary in known language. If you want, you can get more details by looking at the code in `evaluator.py` line 257


In [ ]:
# You can run the following command **sequentially**
# !ls log/ # to get the list of experiment dates
# !ls log/<date>/default/ # to get the list of experiment times, first replace date from the command above
# !ls log/<date>/default/<time>/predictions/<filename> # to get the list of filename, first replace date and time from the commands above

# Replace <date>, <time>, and <filename> from the commands above
# !cat log/<date>/default/<time>/predictions/<filename>

In [ ]:
# Or you can run the following command to download the result to your local environment
from google.colab import files

files.download("log/2021-05-29/default/09-15-25/predictions/extract.1_10.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The other way around is to direct the output to your google drive folder by setting up `--log_dir` parameter on the script. However, by setting up this parameter we won't have the same directory structure format as if we don't set the parameter.